In [13]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [14]:
filename = "../../results_aggr/app_bootstrap_vm.csv"
outfilePrefix = "../../results_aggr/app_bootstrap_vm_plots"
df_values = pd.read_csv(filename, names=["commit", "type", "CI_min", "CI_max"], skiprows=1)
df_values.sort_values(by=["commit"], inplace=True)

In [15]:
for type in df_values.type.unique():
    print(f"Plotting type {type}")

    measurements = df_values.loc[(df_values['type'].str.startswith(type, na=False))]


    #Find % set y-limits
    min = -5
    max = 5
    for minSmall in measurements.CI_min.unique():
        if minSmall < min:
            min = minSmall

    for maxSmall in measurements.CI_max.unique():
        if maxSmall > max:
            max = maxSmall


    sns.set(style="whitegrid")
    axes = plt.axes()
    axes.set_ylim([min, max])

    plot = sns.lineplot(data=measurements, x="commit", y="CI_min", color="royalblue")
    plot = sns.lineplot(data=measurements, x="commit", y="CI_max", color="royalblue")

    x_commits = measurements.commit


    plot.fill_between(x=x_commits, y1=measurements.CI_min, y2=measurements.CI_max, facecolor = 'lightblue')


    plot.set(ylim=(min, max))
    plt.savefig(f'{outfilePrefix}/{type}.png')
    plt.clf()


Plotting type inserts
Plotting type simple queries
Plotting type group-by queries


<Figure size 432x288 with 0 Axes>